In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Hernia"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
23: 144
2: 144
9: 144
22: 144
16: 144
6: 144
2: 144
7: 144
4: 144
3: 144
0: 144
4: 144
6: 144
144: 144
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
139/144 [===========================>..] - ETA: 0s - loss: 6.0955 - leftLayer1_loss: 0.1203 - midLayer1_loss: 1.3410 - rightLayer1_loss: 1.7551 - leftLayer2_loss: 0.1216 - midLayer2_loss: 1.2853 - rightLayer2_loss: 1.4723
Epoch 00006: val_loss improved from 6.27438 to 6.20553, saving model to ./experiments/0.009_weights.h5
144/144 [==============================] - 1s 4ms/step - loss: 6.0935 - leftLayer1_loss: 0.1202 - midLayer1_loss: 1.3403 - rightLayer1_loss: 1.7551 - leftLayer2_loss: 0.1214 - midLayer2_loss: 1.2852 - rightLayer2_loss: 1.4713 - val_loss: 6.2055 - val_leftLayer1_loss: 0.1211 - val_midLayer1_loss: 1.3650 - val_rightLayer1_loss: 1.7496 - val_leftLayer2_loss: 0.1225 - val_midLayer2_loss: 1.2621 - val_rightLayer2_loss: 1.5853
Epoch 7/11
127/144 [=========================>....] - ETA: 0s - loss: 6.0196 - leftLayer1_loss: 0.1195 - midLayer1_loss: 1.3397 - rightLayer1_loss: 1.7254 - leftLayer2_loss: 0.1208 - midLayer2_loss: 1.2996 - rightLayer2_loss: 1.4147
Epoch 

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 144 steps, validate for 41 steps
Epoch 1/11
135/144 [===========================>..] - ETA: 0s - loss: 6.6856 - leftLayer1_loss: 0.1338 - midLayer1_loss: 1.3567 - rightLayer1_loss: 1.9032 - leftLayer2_loss: 0.1386 - midLayer2_loss: 1.2866 - rightLayer2_loss: 1.8666
Epoch 00001: val_loss improved from inf to 6.64389, saving model to ./experiments/0.009999999999999998_weights.h5
144/144 [==============================] - 1s 9ms/step - loss: 6.6795 - leftLayer1_loss: 0.1340 - midLayer1_loss: 1.3549 - rightLayer1_loss: 1.9022 - leftLayer2_loss: 0.1386 - midLayer2_loss: 1.2853 - rightLayer2_loss: 1.8646 - val_loss: 6.6439 - val_leftLayer1_loss: 0.1342 - val_midLayer1_loss: 1.3683 - val_rightLayer1_loss: 1.8907 - val_leftLayer2_loss: 0.1310 - val_midLayer2_loss: 1.2781 - val_rightLayer2_loss: 1.8416
Ep

136/144 [===========================>..] - ETA: 0s - loss: 5.7603 - leftLayer1_loss: 0.1280 - midLayer1_loss: 1.3557 - rightLayer1_loss: 1.6238 - leftLayer2_loss: 0.1232 - midLayer2_loss: 1.2668 - rightLayer2_loss: 1.2627
Epoch 00011: val_loss improved from 6.03084 to 5.97398, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 30s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Hernia: 0.3590994255770282

auprc 0Hernia: 0.0013051105654496872

auroc 1Hernia: 0.18794753844550638

auprc 1Hernia: 0.0010534330687852244

auroc 2Hernia: 0.67441531567743

auprc 2Hernia: 0.003171093367092593

auroc 3Hernia: 0.5586422903760226

auprc 3Hernia: 0.0019535530774762427

auroc 4Hernia: 0.5426484067790842

auprc 4Hernia: 0.0022355430440078195

auroc 5Hernia: 0.7204233843955159

auprc 5Hernia: 0.009089697988380054

mean auroc: 0.5071960602084312

mean auprc: 0.0031347385185319363

max auroc: 0.7204233843955159

max 

Epoch 4/11
139/144 [===========================>..] - ETA: 0s - loss: 6.3829 - leftLayer1_loss: 0.1232 - midLayer1_loss: 1.3648 - rightLayer1_loss: 1.7812 - leftLayer2_loss: 0.1284 - midLayer2_loss: 1.3663 - rightLayer2_loss: 1.6190
Epoch 00004: val_loss improved from 6.43787 to 6.36667, saving model to ./experiments/0.010999999999999998_weights.h5
144/144 [==============================] - 0s 3ms/step - loss: 6.3788 - leftLayer1_loss: 0.1232 - midLayer1_loss: 1.3648 - rightLayer1_loss: 1.7816 - leftLayer2_loss: 0.1281 - midLayer2_loss: 1.3638 - rightLayer2_loss: 1.6172 - val_loss: 6.3667 - val_leftLayer1_loss: 0.1226 - val_midLayer1_loss: 1.3606 - val_rightLayer1_loss: 1.7681 - val_leftLayer2_loss: 0.1250 - val_midLayer2_loss: 1.3004 - val_rightLayer2_loss: 1.6899
Epoch 5/11
137/144 [===========================>..] - ETA: 0s - loss: 6.2620 - leftLayer1_loss: 0.1228 - midLayer1_loss: 1.3696 - rightLayer1_loss: 1.7476 - leftLayer2_loss: 0.1263 - midLayer2_loss: 1.3373 - rightLayer2_loss

...

126/144 [=========================>....] - ETA: 0s - loss: 5.8592 - leftLayer1_loss: 0.1190 - midLayer1_loss: 1.3912 - rightLayer1_loss: 1.5351 - leftLayer2_loss: 0.1194 - midLayer2_loss: 1.4095 - rightLayer2_loss: 1.2850
Epoch 00011: val_loss improved from 5.99441 to 5.93682, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Hernia: 0.695018831971179

auprc 0Hernia: 0.003422756660824259

auroc 1Hernia: 0.5699638034839678

auprc 1Hernia: 0.002175863850537643

auroc 2Hernia: 0.3877902686240858

auprc 2Hernia: 0.001382207741703812

auroc 3Hernia: 0.420517597419031

auprc 3Hernia: 0.001452805436295107

auroc 4Hernia: 0.4518253507467924

auprc 4Hernia: 0.0016516098510292111

auroc 5Hernia: 0.6776766175185183

auprc 5Hernia: 0.0038008485383437965

mean auroc: 0.5337987449605958

mean auprc: 0.002314348679788971

max auroc: 0.695018831971179

max auprc

Epoch 4/11
137/144 [===========================>..] - ETA: 0s - loss: 6.3628 - leftLayer1_loss: 0.1188 - midLayer1_loss: 1.3641 - rightLayer1_loss: 1.7540 - leftLayer2_loss: 0.1168 - midLayer2_loss: 1.3894 - rightLayer2_loss: 1.6198
Epoch 00004: val_loss improved from 6.44476 to 6.37594, saving model to ./experiments/0.012999999999999996_weights.h5
144/144 [==============================] - 0s 3ms/step - loss: 6.3566 - leftLayer1_loss: 0.1187 - midLayer1_loss: 1.3636 - rightLayer1_loss: 1.7528 - leftLayer2_loss: 0.1166 - midLayer2_loss: 1.3871 - rightLayer2_loss: 1.6178 - val_loss: 6.3759 - val_leftLayer1_loss: 0.1201 - val_midLayer1_loss: 1.3728 - val_rightLayer1_loss: 1.7631 - val_leftLayer2_loss: 0.1164 - val_midLayer2_loss: 1.3207 - val_rightLayer2_loss: 1.6829
Epoch 5/11
136/144 [===========================>..] - ETA: 0s - loss: 6.2749 - leftLayer1_loss: 0.1181 - midLayer1_loss: 1.3655 - rightLayer1_loss: 1.7267 - leftLayer2_loss: 0.1141 - midLayer2_loss: 1.3906 - rightLayer2_loss

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 144 steps, validate for 41 steps
Epoch 1/11
138/144 [===========================>..] - ETA: 0s - loss: 6.7306 - leftLayer1_loss: 0.1216 - midLayer1_loss: 1.3220 - rightLayer1_loss: 1.9036 - leftLayer2_loss: 0.1308 - midLayer2_loss: 1.3893 - rightLayer2_loss: 1.8633
Epoch 00001: val_loss improved from inf to 6.58842, saving model to ./experiments/0.013999999999999995_weights.h5
144/144 [==============================] - 1s 9ms/step - loss: 6.7264 - leftLayer1_loss: 0.1216 - midLayer1_loss: 1.3220 - rightLayer1_loss: 1.9038 - leftLayer2_loss: 0.1309 - midLayer2_loss: 1.3864 - rightLayer2_loss: 1.8617 - val_loss: 6.5884 - val_leftLayer1_loss: 0.1216 - val_midLayer1_loss: 1.3273 - val_rightLayer1_loss: 1.9001 - val_leftLayer2_loss: 0.1280 - val_midLayer2_loss: 1.2933 - val_rightLayer2_loss: 1.8181
Ep

135/144 [===========================>..] - ETA: 0s - loss: 5.8380 - leftLayer1_loss: 0.1161 - midLayer1_loss: 1.3253 - rightLayer1_loss: 1.5878 - leftLayer2_loss: 0.1194 - midLayer2_loss: 1.3966 - rightLayer2_loss: 1.2928
Epoch 00011: val_loss improved from 5.98086 to 5.92410, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Hernia: 0.5408157189006637

auprc 0Hernia: 0.0018305346754388858

auroc 1Hernia: 0.6991334741211924

auprc 1Hernia: 0.004885655881880881

auroc 2Hernia: 0.7400411730053104

auprc 2Hernia: 0.0073343574338880484

auroc 3Hernia: 0.4535336263932575

auprc 3Hernia: 0.0018091456286201011

auroc 4Hernia: 0.798949301483802

auprc 4Hernia: 0.0062464399275844115

auroc 5Hernia: 0.5160656598846051

auprc 5Hernia: 0.0018116857820647757

mean auroc: 0.6247564922981385

mean auprc: 0.003986303221579517

max auroc: 0.798949301483802

max a

Epoch 4/11
136/144 [===========================>..] - ETA: 0s - loss: 6.4340 - leftLayer1_loss: 0.1199 - midLayer1_loss: 1.3363 - rightLayer1_loss: 1.7899 - leftLayer2_loss: 0.1227 - midLayer2_loss: 1.3918 - rightLayer2_loss: 1.6733
Epoch 00004: val_loss improved from 6.47569 to 6.40548, saving model to ./experiments/0.014999999999999994_weights.h5
144/144 [==============================] - 0s 3ms/step - loss: 6.4291 - leftLayer1_loss: 0.1198 - midLayer1_loss: 1.3350 - rightLayer1_loss: 1.7882 - leftLayer2_loss: 0.1225 - midLayer2_loss: 1.3932 - rightLayer2_loss: 1.6704 - val_loss: 6.4055 - val_leftLayer1_loss: 0.1207 - val_midLayer1_loss: 1.3197 - val_rightLayer1_loss: 1.7837 - val_leftLayer2_loss: 0.1214 - val_midLayer2_loss: 1.3554 - val_rightLayer2_loss: 1.7047
Epoch 5/11
135/144 [===========================>..] - ETA: 0s - loss: 6.3443 - leftLayer1_loss: 0.1198 - midLayer1_loss: 1.3305 - rightLayer1_loss: 1.7582 - leftLayer2_loss: 0.1205 - midLayer2_loss: 1.4031 - rightLayer2_loss

In [17]:
print(np.max(maxi))

0.8500370046638636
